In [1]:
"""
改代码用于绘制多基因的heatmap
"""

'\n改代码用于绘制多基因的heatmap\n'

In [2]:
import os

from collections import defaultdict

In [47]:
base_path = '/mnt/disk1/xzyao/CancerPNRLE/merge_data'

AML_file = f'{base_path}/Acute Myeloid Leukemia.gene-var.tsv'


go_ancestor_file = '/mnt/disk1/xzyao/CancerPNRLE/result/芷涵GO-HPO分级/go_second_ancestors.txt'

hpo_ancestor_file = '/mnt/disk1/xzyao/CancerPNRLE/result/芷涵GO-HPO分级/hpo_third_ancestors.txt'


fig_save_path = '/mnt/disk1/xzyao/CancerPNRLE/ScientificData投稿富集分析结果/ScientificData_论文绘图/多基因热图绘制'

template_html = f'{fig_save_path}/heatmap-template.html'

In [48]:

def read_obo_level_file(_go_level_file: str):

    go_id_to_second_ancestor = {}
    with open(_go_level_file) as f:
        f.readline()
        for line in f:
            l = line.strip().split('\t')
            if len(l) != 4:
                continue
            go_id = l[0]
            ancestor_id = l[2]
            ancestor_name = l[3]

            if ancestor_name.startswith('obsolete'):
                ancestor_name = ancestor_name.replace('obsolete', '').strip()

            go_id_to_second_ancestor[go_id] = f'{ancestor_id}-{ancestor_name}'

    return go_id_to_second_ancestor


In [49]:
# read ancestor file
go_to_ancestor = read_obo_level_file(go_ancestor_file)
hp_to_ancestor = read_obo_level_file(hpo_ancestor_file)

In [103]:
def heatmap_for_gene_set(_base_path,
                       go_to_ancestor: dict,
                       hpo_to_ancestor: dict,
                       temp_file: str,
                        save_file: str,
                       use_ancestor=False,
                       selected_gene_set=set(),
                        xy_reverse:bool=False):
    
    
    total_bp_set = set()
    cancer_bp_count = defaultdict(int)
    
    print(_base_path)
    
    for file in os.listdir(_base_path):
        print(f'Reading: {file}.')
        file_path = f'{_base_path}/{file}'
        cancer = file.split('.')[0]

        with open(file_path) as f:
            f.readline()
            for line in f:
                l = line.strip().split('\t')

                pmid, sent_id, sentence, rich_sent, snps, alter,\
                gene, events, trigger, include_even, event_genes,\
                go, hpo, mesh, un_norm_bp, basic_score = line.strip().split('\t')

                if gene not in selected_gene_set:
                    continue


                l_go_set = {go_term for go_term in go.split('; ')}
                l_hpo_set = {hpo_term for hpo_term in hpo.split('; ')}
                l_mesh_set = {mesh_term for mesh_term in mesh.split('; ')}


                # get ancestor for go and hpo
                go_ancester_set = set()
                hpo_ancester_set = set()
                for go_term in go.split('; '):
                    if go_term == '-':
                        continue
                    go_id = go_term.split('-')[0]
                    if go_to_ancestor.get(go_id):
                        go_ancester_set.add(go_to_ancestor[go_id])
                go_ancester = '; '.join(go_ancester_set) if go_ancester_set else '-'    

                for hpo_term in hpo.split('; '):
                    if hpo_term == '-':
                        continue
                    hpo_id = hpo_term.split('-')[0]
                    if hpo_to_ancestor.get(hpo_id):
                        hpo_ancester_set.add(hpo_to_ancestor[hpo_id])
                hpo_ancester = '; '.join(hpo_ancester_set) if hpo_ancester_set else '-'


                # start to process data for plot
                if use_ancestor:
                    plot_go_set = set(go_ancester.split('; '))
                    plot_hpo_set = set(hpo_ancester.split('; '))
                else:
                    plot_go_set = set(go.split('; '))
                    plot_hpo_set = set(hpo.split('; '))


                plot_bp_set = plot_go_set|plot_hpo_set


                if '-' in plot_bp_set:
                    plot_bp_set.remove('-')
                if '' in plot_bp_set:
                    plot_bp_set.remove('')

                for bp in plot_bp_set:
                    cancer_bp_count[(cancer, bp)] += 1
                    total_bp_set.add(bp)

    xaxis_set= set()
    yaxis_set = set()
    heatmap_data = []
    for (cancer, bp), value in cancer_bp_count.items():
        
        if xy_reverse:
            heatmap_data.append([cancer, bp, value])
            xaxis_set.add(cancer)
            yaxis_set.add(bp)
        else:
            heatmap_data.append([bp, cancer, value])
            xaxis_set.add(bp)
            yaxis_set.add(cancer)
        
        
    if xy_reverse:
        xaxis_list = list(sorted(xaxis_set, reverse=True))
        yaxis_list = list(sorted(yaxis_set)) 
    else:
        xaxis_list = list(sorted(xaxis_set))
        yaxis_list = list(sorted(yaxis_set, reverse=True)) 

    
    with open(temp_file) as f, open(save_file, 'w') as wf:
        doc = f.read()
        
        doc = doc.replace('xaxis_list', str(xaxis_list))
        doc = doc.replace('yaxis_list', str(yaxis_list))
        doc = doc.replace('heat_data', str(heatmap_data))
        
        wf.write(doc)
    print(f'{save_file} saved.')





In [104]:
def read_gene_file(gene_file: str):
    
    gene_set = set()
    with open(gene_file) as f:
        for line in f:
            gene_set.add(line.strip())
    print(f'gene_num: {len(gene_set):,}')
    print(gene_set)
    return gene_set

In [105]:
gene_path = '/mnt/disk1/xzyao/CancerPNRLE/ScientificData投稿富集分析结果/AML_关联基因'

gene_file_dict = {
    'cancermine-20': f'{gene_path}/CancerMine中AML报道最多了20个基因.tsv',
    'diseases-know': f'{gene_path}/DISEASES中AML-Knowledge的17个基因.tsv',
    'diseases-text': f'{gene_path}/DISEASES中AML-TextMining的20个基因.tsv'
}

In [106]:

xy_reverse = True

for gene_set_name, gene_file_path in gene_file_dict.items():
    print(f'Processing: {gene_set_name}')
    for ancestor in {'ancestor', 'bp'}:
    
        if xy_reverse:
            fig_save_file = f'{fig_save_path}/{gene_set_name}.{ancestor}-reverse.html'
        else:
            fig_save_file = f'{fig_save_path}/{gene_set_name}.{ancestor}.html'

        gene_set = read_gene_file(gene_file_path)
        
        if ancestor == 'ancestor':
            use_ancestor = True
        else:
            use_ancestor = False

        heatmap_for_gene_set(base_path, go_to_ancestor, hp_to_ancestor,
                            template_html, fig_save_file, use_ancestor,
                             gene_set, xy_reverse)
    


Processing: cancermine-20
gene_num: 20
{'HOXA9', 'METTL3', 'RUNX1T1', 'NPM1', 'KMT2A', 'KIT', 'MLLT1', 'FTO', 'RUNX1', 'MYC', 'DNMT3A', 'NRAS', 'METTL14', 'MEIS1', 'MECOM', 'ABL1', 'MLLT3', 'FLT3', 'TP53', 'MYB'}
/mnt/disk1/xzyao/CancerPNRLE/merge_data
Reading: Thymoma.gene-var.tsv.
Reading: Thyroid Papillary Carcinoma.gene-var.tsv.
Reading: Lower Grade Glioma.gene-var.tsv.
Reading: Pancreatic Ductal Adenocarcinoma.gene-var.tsv.
Reading: Uterine Corpus Endometrioid Carcinoma.gene-var.tsv.
Reading: Gastric Adenocarcinoma.gene-var.tsv.
Reading: Cervical Carcinoma.gene-var.tsv.
Reading: Glioblastoma Multiforme.gene-var.tsv.
Reading: Skin Cutaneous Melanoma.gene-var.tsv.
Reading: Lung Squamous Cell Carcinoma.gene-var.tsv.
Reading: Cholangiocarcinoma.gene-var.tsv.
Reading: Head and Neck Squamous Cell Carcinoma.gene-var.tsv.
Reading: Paraganglioma and Pheochromocytoma.gene-var.tsv.
Reading: Hepatocellular Carcinoma.gene-var.tsv.
Reading: Mesothelioma.gene-var.tsv.
Reading: Kidney Clear Cell 

KeyboardInterrupt: 

In [107]:

xy_reverse = True

# ERBB2
for ancestor in {'ancestor', 'bp'}:

    if xy_reverse:
        fig_save_file = f'{fig_save_path}/ERBB2.{ancestor}-reverse.html'
    else:       
        fig_save_file = f'{fig_save_path}/ERBB2.{ancestor}.html'

    gene_set = {'ERBB2'}

    if ancestor == 'ancestor':
        use_ancestor = True
    else:
        use_ancestor = False

    heatmap_for_gene_set(base_path, go_to_ancestor, hp_to_ancestor,
                        template_html, fig_save_file, use_ancestor,
                         gene_set, xy_reverse)



/mnt/disk1/xzyao/CancerPNRLE/merge_data
Reading: Thymoma.gene-var.tsv.
Reading: Thyroid Papillary Carcinoma.gene-var.tsv.
Reading: Lower Grade Glioma.gene-var.tsv.
Reading: Pancreatic Ductal Adenocarcinoma.gene-var.tsv.
Reading: Uterine Corpus Endometrioid Carcinoma.gene-var.tsv.
Reading: Gastric Adenocarcinoma.gene-var.tsv.
Reading: Cervical Carcinoma.gene-var.tsv.
Reading: Glioblastoma Multiforme.gene-var.tsv.
Reading: Skin Cutaneous Melanoma.gene-var.tsv.
Reading: Lung Squamous Cell Carcinoma.gene-var.tsv.
Reading: Cholangiocarcinoma.gene-var.tsv.
Reading: Head and Neck Squamous Cell Carcinoma.gene-var.tsv.
Reading: Paraganglioma and Pheochromocytoma.gene-var.tsv.
Reading: Hepatocellular Carcinoma.gene-var.tsv.
Reading: Mesothelioma.gene-var.tsv.
Reading: Kidney Clear Cell Carcinoma.gene-var.tsv.
Reading: Breast Lobular Carcinoma.gene-var.tsv.
Reading: Sarcoma.gene-var.tsv.
Reading: Colorectal Adenocarcinoma.gene-var.tsv.
Reading: Uterine Carcinosarcoma.gene-var.tsv.
Reading: Uveal 

In [111]:
def heatmap_for_cancer(cancer_db_file,
                       go_to_ancestor: dict,
                       hpo_to_ancestor: dict,
                       temp_file: str,
                        save_file: str,
                       use_ancestor=False,
                      selected_gene_set=set(),
                      xy_reverse: bool=False):
    
    
    total_bp_set = set()
    gene_bp_count = defaultdict(int)
    
    print(f'Processing: {cancer_db_file}')
    with open(cancer_db_file) as f:
        f.readline()
        for line in f:
            l = line.strip().split('\t')

            pmid, sent_id, sentence, rich_sent, snps, alter,\
            gene, events, trigger, include_even, event_genes,\
            go, hpo, mesh, un_norm_bp, basic_score = line.strip().split('\t')
            
            if selected_gene_set:
                if gene not in selected_gene_set:
                    continue
            

            l_go_set = {go_term for go_term in go.split('; ')}
            l_hpo_set = {hpo_term for hpo_term in hpo.split('; ')}
            l_mesh_set = {mesh_term for mesh_term in mesh.split('; ')}


            # get ancestor for go and hpo
            go_ancester_set = set()
            hpo_ancester_set = set()
            for go_term in go.split('; '):
                if go_term == '-':
                    continue
                go_id = go_term.split('-')[0]
                if go_to_ancestor.get(go_id):
                    go_ancester_set.add(go_to_ancestor[go_id])
            go_ancester = '; '.join(go_ancester_set) if go_ancester_set else '-'    

            for hpo_term in hpo.split('; '):
                if hpo_term == '-':
                    continue
                hpo_id = hpo_term.split('-')[0]
                if hpo_to_ancestor.get(hpo_id):
                    hpo_ancester_set.add(hpo_to_ancestor[hpo_id])
            hpo_ancester = '; '.join(hpo_ancester_set) if hpo_ancester_set else '-'


            # start to process data for plot
            if use_ancestor:
                plot_go_set = set(go_ancester.split('; '))
                plot_hpo_set = set(hpo_ancester.split('; '))
            else:
                plot_go_set = set(go.split('; '))
                plot_hpo_set = set(hpo.split('; '))


            plot_bp_set = plot_go_set|plot_hpo_set


            if '-' in plot_bp_set:
                plot_bp_set.remove('-')
            if '' in plot_bp_set:
                plot_bp_set.remove('')

            for bp in plot_bp_set:
                gene_bp_count[(gene, bp)] += 1
                total_bp_set.add(bp)

    xaxis_set= set()
    yaxis_set = set()
    heatmap_data = []
    for (gene, bp), value in gene_bp_count.items():
        
        if xy_reverse:
            heatmap_data.append([gene, bp, value])
            xaxis_set.add(gene)
            yaxis_set.add(bp)
        else:
            heatmap_data.append([bp, gene, value])
            xaxis_set.add(bp)
            yaxis_set.add(gene)
        
    if xy_reverse:
        xaxis_list = list(sorted(xaxis_set, reverse=True))
        yaxis_list = list(sorted(yaxis_set)) 
    else:
        xaxis_list = list(sorted(xaxis_set))
        yaxis_list = list(sorted(yaxis_set, reverse=True)) 

    
    with open(temp_file) as f, open(save_file, 'w') as wf:
        doc = f.read()
        
        doc = doc.replace('xaxis_list', str(xaxis_list))
        doc = doc.replace('yaxis_list', str(yaxis_list))
        doc = doc.replace('heat_data', str(heatmap_data))
        
        wf.write(doc)
    print(f'{save_file} saved.')





In [112]:

xy_reverse = True

for gene_set_name, gene_file_path in gene_file_dict.items():
    print(f'Processing: {gene_set_name}')
    
    gene_set = read_gene_file(gene_file_path)
        
    for ancestor in {'ancestor', 'bp'}:
        
        if xy_reverse:
            fig_save_file = f'{fig_save_path}/AML.{gene_set_name}.{ancestor}-reverse.html'
        else:
            fig_save_file = f'{fig_save_path}/AML.{gene_set_name}.{ancestor}.html'

        if ancestor == 'ancestor':
            use_ancestor = True
        else:
            use_ancestor = False

        heatmap_for_cancer(AML_file, go_to_ancestor, hp_to_ancestor,
                            template_html, fig_save_file, use_ancestor,
                           gene_set, xy_reverse)


Processing: cancermine-20
gene_num: 20
{'HOXA9', 'METTL3', 'RUNX1T1', 'NPM1', 'KMT2A', 'KIT', 'MLLT1', 'FTO', 'RUNX1', 'MYC', 'DNMT3A', 'NRAS', 'METTL14', 'MEIS1', 'MECOM', 'ABL1', 'MLLT3', 'FLT3', 'TP53', 'MYB'}
Processing: /mnt/disk1/xzyao/CancerPNRLE/merge_data/Acute Myeloid Leukemia.gene-var.tsv
/mnt/disk1/xzyao/CancerPNRLE/ScientificData投稿富集分析结果/ScientificData_论文绘图/多基因热图绘制/AML.cancermine-20.bp-reverse.html saved.
Processing: /mnt/disk1/xzyao/CancerPNRLE/merge_data/Acute Myeloid Leukemia.gene-var.tsv
/mnt/disk1/xzyao/CancerPNRLE/ScientificData投稿富集分析结果/ScientificData_论文绘图/多基因热图绘制/AML.cancermine-20.ancestor-reverse.html saved.
Processing: diseases-know
gene_num: 17
{'KRAS', 'RUNX1', 'PML', 'NRAS', 'CBFB', 'AANAT', 'RARA', 'RUNX1T1', 'ABCA3', 'KIT', 'NPM1', 'ZBTB16', 'MYH11', 'STAT5B', 'FLT3', 'AARS', 'NUMA1'}
Processing: /mnt/disk1/xzyao/CancerPNRLE/merge_data/Acute Myeloid Leukemia.gene-var.tsv
/mnt/disk1/xzyao/CancerPNRLE/ScientificData投稿富集分析结果/ScientificData_论文绘图/多基因热图绘制/AML.disea